`pip install pandas`

In [2]:
import pandas as pd

In [4]:
# Load the datasets with index
resources_path = "resources/"

resources = {
    "vg_sales": "video_games_sales_as_at_22_dec_2016.csv",
    "vg_developers": "video_games_developers.csv"
}

vg_sales = pd.read_csv(resources_path + resources["vg_sales"], index_col=0)
vg_developers = pd.read_csv(resources_path + resources["vg_developers"], index_col=0)